### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
movies[:1]

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [3]:
reviews[:1]

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [4]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of rows in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    sorted_reviews = reviews.sort_values(by=order_by, ascending=True)
    training_df = sorted_reviews[:training_size]
    validation_df = sorted_reviews[training_size:training_size + testing_size]

    return training_df, validation_df

In [5]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [6]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [7]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [8]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)
print('Train data head() = \n', train_data_np[:5])

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Train data head() = 
 [[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]
Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.515413
2 		 5.910561
3 		 4.121771
4 		 3.080157
5 		 2.397722
6 		 1.916818
7 		 1.561167
8 		 1.289249
9 		 1.076278
10 		 0.906407
11 		 0.768982
12 		 0.656547
13 		 0.563710
14 		 0.486449
15 		 0.421702
16 		 0.367092
17 		 0.320759
18 		 0.281233
19 		 0.247344
20 		 0.218158
21 		 0.192922
22 		 0.171022
23 		 0.151957
24 		 0.135312
25 		 0.120743
26 		 0.107960
27 		 0.096718
28 		 0.086812
29 		 0.078064
30 		 0.070323
31 		 0.063461
32 		 0.057366
33 		 0.051941
34 		 0.047104
35 		 0.042783
36 		 0.038917
37 		 0.035450
38 		 0.032337
39 		 0.029536
40 		 0.027013
41 		 0.024735
42 		 0.022676
43 		 0.020812
44 		 0.019122
45 		 0.017587
46 		 0.016192
47 		 0.014922
48 		 0.013765
49 		 0.012708
50 

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [17]:
usr_ids = np.array(train_data_df.index)
print('User IDs:\n', usr_ids)
print('\nIn the training data, the row number for user 8 is:\n', np.where(usr_ids == 8)[0][0])

User IDs:
 [    8    46    48 ..., 53943 53962 53966]

In the training data, the row number for user 8 is:
 0


In [18]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Use the training data to create 2 series of User IDs and Movie IDs that match the ordering in the training data
    user_ids_series =  np.array(train_data_df.index)
    movie_ids_series = np.array(train_data_df.columns)
    
    # User the row and Movie column
    user_row =  np.where(user_ids_series == user_id)[0][0]
    movie_col = np.where(movie_ids_series == movie_id)[0][0]
    
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[user_row, :], movie_matrix[:, movie_col])
   
    return pred

In [19]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

6.966375659700871

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [27]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according to the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    movie_name = movies[movies['movie_id'] == movie_id]['movie']
    print('The predicted rating from User {} for Movie "{}" is {}.'.format(
        user_id, list(movie_name)[0], round(prediction, 2)))

In [28]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

The predicted rating from User 8 for Movie "Fantômas - À l'ombre de la guillotine (1913)" is 6.97.


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [38]:
sub_val_df = val_df[:5]
np.array(sub_val_df['movie_id'])

array([1598822,  289879, 1563738, 1458175,  103639])

In [41]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    user_ids, movie_ids = np.array(val_df[:num_preds]['user_id']), np.array(val_df[:num_preds]['movie_id'])
    
    for user_id, movie_id in zip(user_ids, movie_ids):
        actual_rating = list(val_df[(val_df['user_id'] == user_id) & (val_df['movie_id'] == movie_id)]['rating'])[0]
        pred_rating = predict_rating(user_mat, movie_mat, user_id, movie_id)
        print('User {} / Movie {}: actual rating = {} and predicted rating = {}\n'.format(
            user_id, movie_id, actual_rating, pred_rating))
        
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

User 49056 / Movie 1598822: actual rating = 8 and predicted rating = 5.8164529793651205

User 49056 / Movie 289879: actual rating = 9 and predicted rating = 8.24848605267088

User 49056 / Movie 1563738: actual rating = 9 and predicted rating = 7.131181390132628

User 49056 / Movie 1458175: actual rating = 4 and predicted rating = 7.952979286656407

User 28599 / Movie 103639: actual rating = 8 and predicted rating = 8.276202322972312

User 50593 / Movie 1560985: actual rating = 4 and predicted rating = 2.4559316372281064



**The instructor's code is cleaner:**
```
val_users = np.array(val_df['user_id'])
val_movies = np.array(val_df['movie_id'])
val_ratings = np.array(val_df['rating'])
    
for idx in range(num_preds):
    pred = predict_rating(user_mat, movie_mat, val_users[idx], val_movies[idx])
```

In [42]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

User 49056 / Movie 1598822: actual rating = 8 and predicted rating = 5.8164529793651205

User 49056 / Movie 289879: actual rating = 9 and predicted rating = 8.24848605267088

User 49056 / Movie 1563738: actual rating = 9 and predicted rating = 7.131181390132628

User 49056 / Movie 1458175: actual rating = 4 and predicted rating = 7.952979286656407

User 28599 / Movie 103639: actual rating = 8 and predicted rating = 8.276202322972312

User 50593 / Movie 1560985: actual rating = 4 and predicted rating = 2.4559316372281064



IndexError: index 0 is out of bounds for axis 0 with size 0

**A statement about why you think what happened happened.**

The **Cold Start** problem: the validation set contains users and movies that didn't exist in the training set.